In [1]:
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
import time
import sys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import pandas as pd

In [2]:
exeDriver = "chromedriver.exe"
driver = webdriver.Chrome(exeDriver)
print("Iniciando Scraper")

Iniciando Scraper


In [3]:
url = "http://procomun.educalab.es/es"
driver.get(url)

### Busqueda y filtrado por recursos de aprendizaje

In [4]:
content_type = driver.find_element_by_id('edit_type_chosen')
content_type_link = content_type.find_element_by_class_name("chosen-single")
content_type_link.click()

options = content_type.find_elements_by_class_name("active-result")
learning_object = options[1]
learning_object.click()
button = driver.find_element_by_id("new-search")
button.click()
time.sleep(5)

### Obteniendo metadatos articulos

In [5]:
def get_metadata_page():
    URL = driver.current_url
    
    TITULO = driver.find_element_by_xpath('/html/body/section/div/div/div[1]/div/span[5]')
    TITULO = TITULO.text
    
    AUTORES = driver.find_element_by_class_name('author')
    AUTORES = AUTORES.text
    
    FECHA = driver.find_element_by_class_name('detail-ode-publish')
    FECHA = FECHA.text

    PALABRAS_CLAVE = ''
    try:
        general = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[1]/div[1]/div')
        general.click()
        palabras = general.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[1]/div[2]/div/div/div')
        palabras = palabras.text
        palabras_list = palabras.split('\n')
        PALABRAS_CLAVE = palabras_list
    except:
        pass
    
    # Seccion principal
    RESUMEN = ''
    ORIENTACIONES_DIDACTICAS = ''
    LICENCIA = ''
    TIPO_DE_LICENCIA = ''
    USUARIO_FINAL = ''
    CONTEXTO_EDUCATIVO = ''
    seccion_metadatos = driver.find_element_by_id('block-system-main')
    metadatos = seccion_metadatos.find_element_by_xpath('//*[@id="block-system-main"]/div/div/div[2]/div[1]')
    metadatos_txt = metadatos.text
    metadatos_list = metadatos_txt.split('\n')
    for i in range(len(metadatos_list)):
        if (metadatos_list[i] == 'RESUMEN'):
            for j in range(i+1, len(metadatos_list)):
                if ((metadatos_list[j] == 'ORIENTACIONES DIDÁCTICAS') or (metadatos_list[j] == 'LICENCIA')):
                    break
                else:
                    RESUMEN += metadatos_list[j] + ' '
        if (metadatos_list[i] == 'ORIENTACIONES DIDÁCTICAS'):
            for j in range(i+1, len(metadatos_list)):
                if (metadatos_list[j] == 'LICENCIA'):
                    break
                else:
                    ORIENTACIONES_DIDACTICAS += metadatos_list[j] + ' '
        if (metadatos_list[i] == 'LICENCIA'):
            LICENCIA = metadatos_list[i+1]
        if (metadatos_list[i] == 'TIPO DE LICENCIA'):
            TIPO_DE_LICENCIA = metadatos_list[i+1]
        if (metadatos_list[i] == 'USUARIO FINAL'):
            USUARIO_FINAL = metadatos_list[i+1]
        if (metadatos_list[i] == 'CONTEXTO EDUCATIVO'):
            CONTEXTO_EDUCATIVO = metadatos_list[i+1]
        
    # Seccion USO EDUCATIVO 
    USO_EDUCATIVO_EDAD = ''
    USO_EDUCATIVO_IDIOMA_DESTINATARIO = ''
    USO_EDUCATIVO_PROCESO_COGNITIVO = ''
    USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD = ''
    usoEducativo = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[3]/div[1]/div')
    usoEducativo.click()
    elementos = usoEducativo.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[3]')
    elementos = elementos.text
    elementos_list = elementos.split('\n')
    length = len(elementos_list)
    if (length > 1):
        for i in range(1, length):
            if (elementos_list[i] == 'EDAD'):
                USO_EDUCATIVO_EDAD = elementos_list[i+1]
            if (elementos_list[i] == 'IDIOMA DESTINATARIO'):
                USO_EDUCATIVO_IDIOMA_DESTINATARIO = elementos_list[i+1]
            if (elementos_list[i] == 'PROCESO COGNITIVO'):
                USO_EDUCATIVO_PROCESO_COGNITIVO = elementos_list[i+1]
            if (elementos_list[i] == 'TIPO DE INTERACTIVIDAD'):
                USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD = elementos_list[i+1]
                
    # Seccion tecnica
    TECNICA_PLATAFORMA = ''
    try:
        tecnica = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[2]/div[1]/div')
        tecnica.click()
        elementosTecnica = tecnica.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[2]/div[2]/div/div/div')
        TECNICA_PLATAFORMA = elementosTecnica.text
    except:
        pass
    
    # Seccion clasificacion
    CLASIFICACION_AREA_DE_CONOCIMIENTO = ''
    CLASIFICACION_CONTEXTO_EDUCATIVO = ''
    CLASIFICACION_TIPO_DE_RECURSO = ''
    clasificacion = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[4]/div[1]/div')
    clasificacion.click()
    elemClasificacion = clasificacion.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[4]')
    elemClasificacion = elemClasificacion.text
    elemClasificacion_list = elemClasificacion.split('\n')
    length = len(elemClasificacion_list)
    if (length > 1):
        for i in range(1, length):
            if (elemClasificacion_list[i] == 'TIPO DE RECURSO'):
                CLASIFICACION_TIPO_DE_RECURSO = elemClasificacion_list[i+1]
            if (elemClasificacion_list[i] == 'ÁREA DE CONOCIMIENTO'):
                CLASIFICACION_AREA_DE_CONOCIMIENTO = elemClasificacion_list[i+1]
            if (elemClasificacion_list[i] == 'CONTEXTO EDUCATIVO'):
                CLASIFICACION_CONTEXTO_EDUCATIVO = elemClasificacion_list[i+1]

    # Agregando a df
    diccionario = {'TITULO': TITULO, 'PALABRAS_CLAVE': PALABRAS_CLAVE, 'RESUMEN': RESUMEN, 
                   'ORIENTACIONES_DIDACTICAS': ORIENTACIONES_DIDACTICAS, 'LICENCIA': LICENCIA, 
                   'TIPO_DE_LICENCIA': TIPO_DE_LICENCIA, 'AUTORES': AUTORES, 'USUARIO_FINAL': USUARIO_FINAL, 
                   'CONTEXTO_EDUCATIVO': CONTEXTO_EDUCATIVO, 'FECHA': FECHA, 'USO_EDUCATIVO_EDAD': USO_EDUCATIVO_EDAD, 
                   'USO_EDUCATIVO_IDIOMA_DESTINATARIO': USO_EDUCATIVO_IDIOMA_DESTINATARIO, 'USO_EDUCATIVO_PROCESO_COGNITIVO': USO_EDUCATIVO_PROCESO_COGNITIVO,
                   'USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD': USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD, 'URL': URL, 'TECNICA_PLATAFORMA': TECNICA_PLATAFORMA,
                   'CLASIFICACION_AREA_DE_CONOCIMIENTO': CLASIFICACION_AREA_DE_CONOCIMIENTO, 'CLASIFICACION_CONTEXTO_EDUCATIVO': CLASIFICACION_CONTEXTO_EDUCATIVO, 
                   'CLASIFICACION_TIPO_DE_RECURSO': CLASIFICACION_TIPO_DE_RECURSO}
    return diccionario

In [6]:
columns = ['TITULO', 'PALABRAS_CLAVE', 'AUTORES', 'FECHA', 'RESUMEN', 'ORIENTACIONES_DIDACTICAS', 
           'LICENCIA', 'TIPO_DE_LICENCIA', 'USUARIO_FINAL', 'CONTEXTO_EDUCATIVO', 'USO_EDUCATIVO_EDAD',
           'USO_EDUCATIVO_IDIOMA_DESTINATARIO', 'USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD', 
           'USO_EDUCATIVO_PROCESO_COGNITIVO','TECNICA_PLATAFORMA', 'CLASIFICACION_AREA_DE_CONOCIMIENTO', 
           'CLASIFICACION_CONTEXTO_EDUCATIVO', 'CLASIFICACION_TIPO_DE_RECURSO', 'URL']
df = pd.DataFrame(columns=columns)

for i in range(3): # Itera sobre paginas
    for j in range(7, 11): # Itera sobre articulos
        articulo1 = driver.find_element_by_xpath('/html/body/section/div/div/div/section/div/section/div[2]/ul/li[' + str(j) + ']/div[1]/h2/a')
        articulo1.click()
        driver.switch_to.window(driver.window_handles[-1])

        diccionario = get_metadata_page()   
        df = df.append(diccionario, ignore_index=True)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    next_page = driver.find_element_by_class_name("pager-next")
    next_page.click()
    time.sleep(5)

df

,TITULO,PALABRAS_CLAVE,AUTORES,FECHA,RESUMEN,ORIENTACIONES_DIDACTICAS,LICENCIA,TIPO_DE_LICENCIA,USUARIO_FINAL,CONTEXTO_EDUCATIVO,USO_EDUCATIVO_EDAD,USO_EDUCATIVO_IDIOMA_DESTINATARIO,USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD,USO_EDUCATIVO_PROCESO_COGNITIVO,TECNICA_PLATAFORMA,CLASIFICACION_AREA_DE_CONOCIMIENTO,CLASIFICACION_CONTEXTO_EDUCATIVO,CLASIFICACION_TIPO_DE_RECURSO,URL
0,les_veus_dahir_i_davui,"[democracia, mujer, feminismo, movimientos soc...",FELIPE PÉREZ FERRER,26/09/2019,Breve proyecto que trabaja la memoria democrát...,,Creative Commons: Reconocimiento - No comercia...,Universal,alumno con altas capacidades grupal,Aula Laboratorio Mixta Semipresencial,15-16,valenciano,,,,Historia,15 - 16 / Cuarto curso,"Texto narrativo, Basado en proyectos",http://procomun.educalab.es/es/ode/view/155912...
1,actividades grupos interactivos,"[Favorecer la interacción social, al oralidad,...",irene gonzalez collado,23/09/2019,Batería de actividades para desarrollar en los...,,Creative Commons: Reconocimiento - Compartir i...,Universal,tutor grupal,Aula Laboratorio Mixta Presencial,0-6 6-12,español (o castellano),,,,"Educación Ambiental, Lengua, Lenguas Extranjer...","Educación Infantil, Educación Primaria",Juego didáctico,http://procomun.educalab.es/es/ode/view/155912...
2,¿Truco o trato? ¿Calabaza o nabo?,"[Halloween, Epidemias, clima, Cultivos, Cienci...",Abigail Jiménez,23/09/2019,Trabajo propuesto para el Día de Halloween sob...,,Creative Commons: Reconocimiento - Compartir i...,Universal,alumno individual,Aula Laboratorio Mixta Presencial,9-10,español (o castellano),,,,"Ciencias Biológícas, Educación Ambiental, Hist...",9 - 10 años / Cuarto curso,"Texto narrativo, Basado en proyectos",http://procomun.educalab.es/es/ode/view/155912...
3,Actividades para el tema de Conocimiento Cient...,"[método científico, gráficas, ecuaciones, Camb...",Abigail Jiménez,23/09/2019,Actividades para el tema de Conocimiento Cient...,,Creative Commons: Reconocimiento - Compartir i...,Universal,alumno individual,Aula Laboratorio Mixta Presencial,7-8,español (o castellano),,,,Física,7 - 8 años / Segundo curso,"Texto narrativo, Ejercicio o problema cerrado",http://procomun.educalab.es/es/ode/view/155912...
4,Animación Huesos OJIGI,"[3D, software, LIBRE, Animación, Blender]",ANTONIO MONTES DE MIGUEL,09/09/2019,Principios básicos de Animación 3D con Softwar...,Conceptos básicos Blender,Creative Commons: Reconocimiento - Compartir i...,Universal,alumno individual,Aula Laboratorio Mixta Presencial,,español (o castellano),,,,"Artes Escénicas, Artes Plásticas, Diseño, Tecn...",Ciclo formativo grado superior (más de 18 años...,Ejercicio o problema cerrado,http://procomun.educalab.es/es/ode/view/155912...
5,Materiales. Estructura interna y propiedades,,Adolfo Amella Santolaria,06/09/2019,2º Bachillerato. Bloque Materiales. Tema 1.,,Creative Commons: Reconocimiento - Compartir i...,Universal,alumno individual,Aula Laboratorio Mixta Presencial,16-18,español (o castellano),,,,Tecnologías,Bachillerato,Lección magistral,http://procomun.educalab.es/es/ode/view/155912...
6,Teacher training models comparison (Comparativ...,"[Sharetrain, erasmus, teacher training, Formac...",Olga Gómez González,04/09/2019,Teacher training models comparison in 4 Europe...,,Creative Commons: Reconocimiento - Sin obra de...,Universal,alumno individual,Aula Laboratorio Mixta Presencial,,español (o castellano),,,,Educación Intercultural,Formación del Profesorado,"Texto narrativo, Lección magistral",http://procomun.educalab.es/es/ode/view/155912...
7,comparison_of_educative_systems,"[Sharetrain, educative system, educación, prof...",Olga Gómez González,04/09/2019,Comparison of Educative System of 4 European C...,,Creative Commons: Reconocimiento - Sin obra de...,Universal,alumno individual,Aula Laboratorio Mixta Presencial,,español (o castellano),,,,Educación Intercultural,Formación del Profesorado,"Texto narrativo, Lección magistral",http://procomun.educalab.es/es/ode/view/155912...
8,M2 Components 

In [11]:
df.to_csv("data.csv", sep = ';', encoding = 'ISO-8859-1')